# North Carolina 2023 Redistricting Asian American Analysis

We received a request for an analysis of the Asian American population in North Carolina's newly passed maps. Broadly speaking, the analysis includes
1. where each district leans overall (safe D, safe R, lean D, lean R)
2. AAPI CVAP (raw and percentages)
3. where each district leans for just AAPI voters (safe D, safe R, lean D, lean R) 

For the above, we look at voter registration among self-identified Asian voters as a proxy for (3).

File sources:
- Voterfile data is sourced from an individual-level voter file from L2, not posted on RDH website.
- Election results come from the [RDH website](https://redistrictingdatahub.org/dataset/2020-presidential-democratic-republican-vote-share-on-nationwide-2020-census-blocks/).
- CVAP data from the [RDH website](https://redistrictingdatahub.org/dataset/north-carolina-cvap-data-disaggregated-to-the-2020-block-level-2021/).

In [1]:
import geopandas as gp
import pandas as pd
import os

In [2]:
def add_info(df):
    
    df["Rep_Share"] = df["G20PRERTRU"] / (df["G20PRERTRU"] + df["G20PREDBID"])
    
    df["CVAP_ASN21_pct"] = df["CVAP_ASN21"] / df["CVAP_TOT21"]
    
    return df

## Load raw files

In [3]:
nc_sldl_shp = gp.read_file("./raw-from-source/SL 2023-149 - Shapefile.zip")
nc_sldu_shp = gp.read_file("./raw-from-source/SL 2023-146 - Shapefile.zip")
nc_cong_shp = gp.read_file("./raw-from-source/SL 2023-145 - Shapefile.zip")

In [4]:
VOTE_STRING = "/Users/peterhorton/Documents/RDH/raw_data/elections/national_block_2020_pres_results/national_block_2020_pres_results.csv"

In [5]:
national_votes = pd.read_csv(VOTE_STRING)
nc_votes = national_votes[national_votes["STATEAB"]=="NC"]
nc_cvap = pd.read_csv("./raw-from-source/nc_cvap_2021_2020_b/nc_cvap_2021_2020_b.csv")

nc_sldl = pd.read_csv("./raw-from-source/SL 2023-149 - Block Assignment File.zip")
nc_sldu = pd.read_csv("./raw-from-source/SL 2023-146 - Block Assignment File.zip")
nc_cong = pd.read_csv("./raw-from-source/SL 2023-145 - Block Assignment File.zip")

## Clean Files

In [6]:
nc_sldl.rename(columns = {"District":"SLDL"}, inplace = True)
nc_sldu.rename(columns = {"District":"SLDU"}, inplace = True)
nc_cong.rename(columns = {"District":"CONG"}, inplace = True)

In [7]:
nc_sldl.index = nc_sldl["Block"]
nc_sldu.index = nc_sldu["Block"]
nc_cong.index = nc_cong["Block"]

In [8]:
nc_sldl.drop(["Block"], axis = 1, inplace = True)
nc_sldu.drop(["Block"], axis = 1, inplace = True)
nc_cong.drop(["Block"], axis = 1, inplace = True)

## Join Files

In [9]:
comb = nc_sldl.join([nc_sldu, nc_cong])
comb = pd.merge(comb, nc_votes, left_on = "Block", right_on = "GEOID20", indicator = True)
comb = pd.merge(comb, nc_cvap, on = "GEOID20", indicator = "merge_3")

## Group and Run Computations

In [10]:
nc_sldl = comb.groupby("SLDL").sum()
nc_sldu = comb.groupby("SLDU").sum()
nc_cong = comb.groupby("CONG").sum()

/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_60715/301357391.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  nc_sldl = comb.groupby("SLDL").sum()
/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_60715/301357391.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  nc_sldu = comb.groupby("SLDU").sum()
/var/folders/1t/0q4w6hm92mg_zxd84dfxmq3m0000gn/T/ipykernel_60715/301357391.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which shoul

In [11]:
nc_sldl.reset_index(inplace = True, drop = False)
nc_sldu.reset_index(inplace = True, drop = False)
nc_cong.reset_index(inplace = True, drop = False)

In [12]:
add_info(nc_sldl)
add_info(nc_sldu)
add_info(nc_cong)


,CONG,SLDL,SLDU,GEOID20,COUNTYFP20,TOT_POP,VAP_MOD,G20PREDBID,G20PRELJOR,G20PRERTRU,...,CVAP_AIA21,CVAP_ASN21,CVAP_NHP21,CVAP_2OM21,CVAP_BLW21,CVAP_AIW21,CVAP_ASW21,CVAP_AIB21,Rep_Share,CVAP_ASN21_pct
0,1,409592,100546,8890160802000302377,889004823,745670,576109,195535.03,2155.53,189035.87,...,8184.75,3996.64,398.02,1135.89,2843.20,2970.31,845.95,1380.83,0.491550,0.007074
1,2,366495,141025,3428278006733084093,342827260,745670,580045,274374.49,4729.59,129373.77,...,4688.57,21414.34,282.99,2522.75,3167.83,1905.58,2782.82,1407.04,0.320432,0.040395
2,3,536089,103833,8236621389952688149,823649474,745671,582489,148669.27,3943.30,211946.86,...,6043.63,8944.71,580.05,2398.99,4655.18,3485.13,2226.06,633.96,0.587735,0.015925
3,4,374130,216733,4045945544067947196,404594258,745671,571483,303300.21,3982.86,109913.03,...,4361.94,35502.23,239.92,1755.30,3411.30,1981.47,2523.33,1047.00,0.265996,0.070139
4,5,1529547,675618,7043306946899103272,704322734,745671,592794,167158.70,2919.90,228749.67,...,4624.76,8741.79,405.92,1060.00,3137.77,2302.68,737.23,610.58,0.577784,0.015321
5,6,1126561,462714,5573359514285658600,557335323,745671,576195,157030.82,3259.44,218930.89,...,5228.46,11087.72,122.95,1167.64,2783.40,3068.76,1065.42,793.50,0.582322,0.020632
6,7,571631,228040,7373837637917640502,737376701,745671,589268,182078.82,3700.84,227836.28,...,19191.10,6813.14,227.06,1533.16,3443.98,4066.63,1700.10,1466.03,0.555813,0.011997
7,8,1190198,583453,7053528496152303891,705343315,745671,555461,160801.32,3072.63,231207.40,...,36552.09,13234.23,55.95,801.02,3177.48,2807.59,1217.41,700.49,0.589802,0.024855
8,9,1106453,442052,6878887521686476484,687884061,745671,576688,157590.48,3258.76,209218.46,...,10047.48,9703.06,687.04,2476.90,3740.03,3793.25,1884.36,1422.65,0.570374,0.017846
9,10,1422776,635354,6121156565149797430,612115153,745670,579846,164107.30,3276.75,227816.72,...,3748.76,9957.49,153.99,1246.11,2846.14,2274.63,1077.95,609.01,0.581278,0.018352


In [13]:
nc_sldl.sort_values("CVAP_ASN21_pct", ascending = False, inplace = True)

## Incorporate Voter File Data

In [14]:
vf_data_cong = pd.read_csv("./raw-from-source/new_map_stats_by_party_nc/cong_reg_voters_party_new_maps_nc.csv")
vf_data_sldl = pd.read_csv("./raw-from-source/new_map_stats_by_party_nc/sldl_reg_voters_party_new_maps_nc.csv")
vf_data_sldu = pd.read_csv("./raw-from-source/new_map_stats_by_party_nc/sldu_reg_voters_party_new_maps_nc.csv")

In [15]:
nc_sldl["SLDL"] = nc_sldl["SLDL"].astype(str).str.zfill(3)
nc_sldu["SLDU"] = nc_sldu["SLDU"].astype(str).str.zfill(2)
nc_cong["CONG"] = nc_cong["CONG"].astype(str).str.zfill(2)

In [16]:
sldl_export = pd.merge(vf_data_sldl, nc_sldl, how = "left", on = "SLDL")
sldu_export = pd.merge(vf_data_sldu, nc_sldu, how = "left", on = "SLDU")
cong_export = pd.merge(vf_data_cong, nc_cong, how = "left", on = "CONG")

## Final Clean and Export

In [17]:
sldl_export.drop([i for i in sldl_export.columns if i[0:2]=="P0"], axis = 1, inplace = True)

In [18]:
if not os.path.exists("./nc_2023_plans_asian_american_analysis"):
    os.mkdir("./nc_2023_plans_asian_american_analysis")

In [19]:
sldl_export = sldl_export[["SLDL", "Rep_Share", 'CVAP_TOT21','CVAP_ASN21',"CVAP_ASN21_pct",'ASN_REG','ASN_NON','ASN_REP','ASN_DEM','ASN_GRE', 'ASN_LIB']]
sldu_export = sldu_export[["SLDU", "Rep_Share", 'CVAP_TOT21','CVAP_ASN21',"CVAP_ASN21_pct",'ASN_REG','ASN_NON','ASN_REP','ASN_DEM','ASN_GRE', 'ASN_LIB']]
cong_export = cong_export[["CONG", "Rep_Share", 'CVAP_TOT21','CVAP_ASN21',"CVAP_ASN21_pct",'ASN_REG','ASN_NON','ASN_REP','ASN_DEM','ASN_GRE', 'ASN_LIB']]


In [23]:
sldu_export 

,DISTRICT,Rep_Share,CVAP_TOT21,CVAP_ASN21,CVAP_ASN21_pct,ASN_REG,ASN_NON,ASN_REP,ASN_DEM,ASN_GRE,ASN_LIB
0,01,0.544613,157289.96,1344.99,0.008551,453,213,120,117,0,3
1,02,0.559338,159234.98,1106.96,0.006952,490,220,137,130,0,3
2,03,0.583016,153889.47,2101.96,0.013659,1106,458,359,286,1,2
3,04,0.530600,156720.08,1572.00,0.010031,683,259,209,209,0,6
4,05,0.432067,166760.02,2104.87,0.012622,897,459,165,266,0,7
5,06,0.655036,150045.18,3766.92,0.025105,1272,574,435,235,0,28
6,07,0.526396,156228.51,2162.95,0.013845,1094,573,251,259,0,11
7,08,0.581068,173600.94,1331.23,0.007668,580,279,158,140,0,3
8,09,0.613240,147241.10,602.11,0.004089,329,155,95,79,0,0
9,10,0.623596,146985.03,1367.83,0.009306,738,367,189,179,0,3


In [22]:
cong_export

,DISTRICT,Rep_Share,CVAP_TOT21,CVAP_ASN21,CVAP_ASN21_pct,ASN_REG,ASN_NON,ASN_REP,ASN_DEM,ASN_GRE,ASN_LIB
0,01,0.491550,564995.62,3996.64,0.007074,1770,724,456,577,0,13
1,02,0.320432,530123.34,21414.34,0.040395,13177,7088,1845,4191,3,50
2,03,0.587735,561679.01,8944.71,0.015925,3658,1677,1086,856,1,38
3,04,0.265996,506171.54,35502.23,0.070139,26239,14664,2134,9366,3,72
4,05,0.577784,570579.51,8741.79,0.015321,4869,2365,1161,1319,0,24
5,06,0.582322,537402.15,11087.72,0.020632,7310,3672,1419,2178,2,39
6,07,0.555813,567884.29,6813.14,0.011997,3559,1772,911,854,1,21
7,08,0.589802,532466.77,13234.23,0.024855,9019,4919,1643,2414,0,43
8,09,0.570374,543724.25,9703.06,0.017846,4654,2222,1146,1259,0,27
9,10,0.581278,542592.90,9957.49,0.018352,5696,3087,1050,1534,0,25


In [20]:
sldl_export.rename(columns = {"SLDL":"DISTRICT"}, inplace = True)
sldu_export.rename(columns = {"SLDU":"DISTRICT"}, inplace = True)
cong_export.rename(columns = {"CONG":"DISTRICT"}, inplace = True)

In [21]:
sldl_export.to_csv("./nc_2023_plans_asian_american_analysis/sldl_analysis.csv", index = False)
sldu_export.to_csv("./nc_2023_plans_asian_american_analysis/sldu_analysis.csv", index = False)
cong_export.to_csv("./nc_2023_plans_asian_american_analysis/cong_analysis.csv", index = False)